# 实战项目 3：Smart Beta 投资组合与投资组合优化

## 概述


Smart beta 的含义很丰富，不过在实践中，如果我们使用某个指数的股票池并应用市值加权之外的其他加权方法，则属于 smart beta 基金。Smart Beta 投资组合通常使投资者能够暴露于一种或多种市场特性（或因素），这些特性能够预测价格并使投资者能够广泛分散暴露于特定的市场的风险。Smart Beta 投资组合通常以动量、收益质量、低波动率和股息或其中几种组合形式为目标。Smart Beta 投资组合一般很少再平衡，遵循相对简单的规则或算法（被动管理）。这些类型的基金模型很少有变化，如果是美国共同基金或 ETF，则改变模型需要向美国证券交易委员会提交大量资料。Smart Beta portfolios 通常只做多，不做空股票。

相反，完全侧重于 alpha 的基金可能会使用多个模型或算法来创建投资组合。投资组合经理会决定是否升级或更改模型类型，以及多久再平衡一次投资组合，从而相对于股票基准来说最大化业绩。经理可能会决定做空投资组合中的股票。

假设你是一名投资组合经理，想要尝试不同的投资组合加权方法。

一种设计投资组合的方法是查看某些基于过去走势的会计衡量值（基本面）并寻找业绩更好的股票。


例如，首先假设发放股息的股票往往比不发放股息的股票业绩好。并非所有公司都这样。例如，Apple 不发放股息，但是历史业绩很好。关于发放股息股票的假设如下所示：

定期发放股息的公司在分配可用现金时可能更谨慎，并且表明他们更关心股东的利益。例如，CEO 可能决定将现金重新投资到收益率低的宠物项目上。或者，CEO 可能会做出分析，发现重新投资公司产品产生的收益率比分散化投资组合的低，所以认为（以股息形式）向股东发放现金对股东更有利。根据这种假设，股息不仅能体现公司的业绩（收益和现金流），而且表明公司以股东利益为宗旨。当然，需要检验这种假设在实践中是否成立。


你还可能提出其他假设，根据该假设设计一个投资组合，然后将其变成 ETF。你发现投资者希望投资被动 beta 基金，但是希望风险暴露水平更低（波动性更小）。如果基金的波动性很低，但是收益率依然与指数的相似，那么投资时间范围更短（更容易规避风险）的投资者更喜欢这种基金。

提议投资组合的目标是设计一个紧密跟踪指数的投资组合，同时能够最小化投资组合方差。此外，如果该投资组合的收益率与指数的差不多，但是波动率更小，那么它的风险调整收益率更高（相同的收益率，波动性更小）。

我们可以通过以下两种方法（以及其他方法）设计 Smart Beta ETF：替代加权和最低波动性 ETF。


## 项目说明
每个问题都包含需要实现的函数和如何实现该函数的说明。`# TODO` 注释表示需要实现的函数部分。实现函数后，请运行单元格并用我们提供的单元测试检测代码。对于每个问题，我们都在  `project_tests`  软件包中提供了一个或多个单元测试。这些单元测试不会指出你的答案是否正确，但是会提醒你重大错误。当你提交项目后，我们会检查你的答案是否正确。

## 软件包
在实现函数时，你只需使用在教室中用到的软件包，例如 [Pandas][1] 和 [Numpy][2]。我们会为你导入这些软件包。建议不要添加任何导入语句，否则审阅者可能无法运行你的代码。

我们还导入了以下其他软件包：`helper`、`project_helper` 和 `project_tests`。这些是专门帮助你解决问题的自定义软件包。`helper` 和 `project_helper` 模块包含实用函数和图形函数。 `project_tests` 包含所有问题的单元测试。
### 安装软件包

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### 加载软件包

In [ ]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## 市场数据
### 加载数据
对于股票池，我们将选择大额股票。这个股票池的流动性很高，所以使用它。

In [ ]:
df = pd.read_csv('../../data/project_3/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### 查看数据
我们看看收盘价矩阵，了解这两个二维矩阵看起来如何。

In [ ]:
project_helper.print_dataframe(close)

# 第一部分：Smart Beta 投资组合
在第一部分，你将构建一个投资组合，并根据股息收益率选择投资组合权重。这样的投资组合可以添加到 smart beta ETF 中。你需要将此投资组合与市值加权的指数进行比较，看看它的业绩如何。

注意，在实践中，你很可能会从数据供应商（例如创建指数的公司，比如 MSCI、FTSE、标普）那获得指数权重，但是对于这道练习，我们将模拟市值加权的指数。

## 指数权重
我们将使用的指数基于大额股票。请实现 `generate_dollar_volume_weights` 以生成该指数的权重。对于每个日期，请根据该日期的成交额生成权重。例如，假设下面是收盘价和成交数据：

In [ ]:
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...

函数 `generate_dollar_volume_weights` 创建的权重应该如下所示：

In [ ]:
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...

In [ ]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function

    return None

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

### 查看数据
使用 `generate_dollar_volume_weights` 生成指数权重并通过热图查看这些权重。

In [ ]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

## 投资组合权重
获得指数权重后，我们根据股息选择投资组合权重。通常，你会根据往迹股息收益率计算权重，但是我们将简化为计算一段时间的总股息收益率。

实现 `calculate_dividend_weights` 以根据每支股票在一段时间的总股息收益率返回该股票的权重。类似于生成指数权重，但使用的是股息数据。
例如，假设以下是 `dividends` 数据：

In [ ]:
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...

函数 `calculate_dividend_weights` 创建的权重应该如下所示：

In [ ]:
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...

In [ ]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function

    return None

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

### 查看数据
与指数权重一样，我们生成 ETF 权重并通过热图查看这些权重。

In [ ]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

## 收益率
实现 `generate_returns` 以针对股价数据中的所有股票和日期生成收益率数据。你可能注意到了，我们实现的是收益率，而不是对数收益率。因为我们不处理波动性，所以不用使用对数收益率。

In [ ]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function

    return None

project_tests.test_generate_returns(generate_returns)

### 查看数据
使用 `generate_returns` 生成收盘价收益率并通过热图查看这些收益率。

In [ ]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

## 加权收益率
计算每支股票的收益率后，我们利用这些收益率计算指数或 ETF 的收益率。实现 `generate_weighted_returns` 以使用收益率和权重创建加权收益率。

In [ ]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function

    return None

project_tests.test_generate_weighted_returns(generate_weighted_returns)

### 查看数据
使用 `generate_weighted_returns` 生成 ETF 和指数收益率并通过热图查看它们。

In [ ]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## 累积收益率
为了比较 ETF 和指数的业绩，我们将计算跟踪误差。首先需要计算指数和 ETF 累积收益率。实现 `calculate_cumulative_returns` 以根据给定收益率生成一段时间的累积收益率。

In [ ]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
    
    return None

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

### 查看数据
使用 `calculate_cumulative_returns` 生成 ETF 和指数累积收益率并比较二者。

In [ ]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

## 跟踪误差
为了检查 smart beta 投资组合的业绩，我们可以计算相对于指数的年化跟踪误差。实现 `tracking_error` 以返回 ETF 和基准之间的跟踪误差。

我们将使用以下年化跟踪误差公式：
$$ TE = \sqrt{252} \* SampleStdev(r\_p - r\_b) $$

其中 $ r\_p $ 是投资组合或 ETF 收益率，$ r\_b $ 是基准收益率。

_注意：在计算样本标准差时，自由度是 1，这是默认值。_

In [ ]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function

    return None

project_tests.test_tracking_error(tracking_error)

### 查看数据
使用 `tracking_error` 生成跟踪误差。

In [ ]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

# 第二部分：投资组合优化

下面创建第二个投资组合。我们依然使用市值加权指数，但是与在第一部分创建的股息加权投资组合无关。

我们希望最小化投资组合方差以及紧密跟踪市值加权指数。换句话说，我们希望最小化投资组合权重与指数权重之间的距离。

$Minimize \left [ \sigma^2\_p + \lambda \sqrt{\sum\_{1}^{m}(weight\_i - indexWeight\_i)^2} \right  ]$ 其中 $m$ 是投资组合中的股票数量，$\lambda$ 是一个缩放因子，你可以随意选择。

为何这么做？投资者评估基金的一种方式是查看基金跟踪指数的效果。为了提高基金业绩，依然希望基金与指数的差异在一定范围内。基金跟踪基准业绩的一种方式是让资产权重与指数权重相似。我们预计，如果基金股票与基准股票一样，并且每支股票的权重与基准的也一样，那么基金的收益率应该与基准的相似。通过最小化投资组合风险和投资组合与基准权重之间距离的线性组合，我们希望平衡最小化投资组合方差这一目标与指数跟踪目标。 


## 协方差
实现 `get_covariance_returns` 以计算 `returns` 的协方差。我们将使用它计算投资组合方差。

如果有 $m$ 支股票，协方差矩阵是一个 $m \times m$ 矩阵，其中包含每对股票之间的协方差。我们可以使用 [`Numpy.cov`][3] 获得协方差。向其传入一个二维矩阵，每行代表股票，每列代表同一时期的观察值。对于任何 `NaN` 值，可以使用 [`DataFrame.fillna`][4] 函数将其替换为 0。

协方差矩阵 $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\\\
\end{bmatrix}$

In [ ]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    
    return None

project_tests.test_get_covariance_returns(get_covariance_returns)

### 查看数据
下面查看 `get_covariance_returns` 生成的协方差。

In [ ]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

### 投资组合方差
我们可以将投资组合方差写成 $\sigma^2\_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

$\mathbf{x^T} \mathbf{P} \mathbf{x}$ 称为二次型 。
我们可以使用 cvxpy 函数 `quad_form(x,P)` 获得二次型。

### 与指数权重之间的距离
我们希望获得能紧密跟踪指数的投资组合权重。所以我们希望最小化二者之间的距离。
根据勾股定理，在 x,y 平面中的两点之间的距离等于 x 和 y 距离的平方和再求平方根。将此定理延伸到任何维数后变成 L2 范数。所以：$\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$ ， 还可以写成 $\left \| \mathbf{x} - \mathbf{index} \right \|_2$。有一个 cvxpy 函数，叫做 [norm()][5]
`norm(x, p=2, axis=None)`。默认情况下就是计算 L2 范数，所以需要传入一个参数，即投资组合权重与指数权重之间的差异。

### 目标函数
我们想要最小化投资组合方差以及投资组合权重与指数权重之间的距离。还需要选择一个 `scale` 常量，即表达式中的 $\lambda$。

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|\_2$


它表示相对于最小化投资组合方差而言，最小化与指数之间差异的优先级是多少。如果 `scale` ($\lambda$) 的值很大，你认为最小化差异的优先级更高，还是最小化方差的优先级更高？

我们可以使用 cvxpy  `objective = cvx.Minimize()` 得出目标函数。你认为应该向此函数传入什么？



### 约束条件
还可以使用列表定义约束条件。例如，如果你希望权重之和为 1，则 $\sum\_{1}^{n}x = 1$。并且可能需要只做多头寸，也就是不做空，所以权重不能为负。所以对所有 $i$ 来说，$x\_i \>0 $，可以将变量另存为 `[x >= 0, sum(x) == 1]`，其中 x 是用 `cvx.Variable()` 创建的。

### 优化
设定目标函数和约束条件后，我们可以求解 $\mathbf{x}$ 的值。
cvxpy 具有构造函数 `Problem(objective, constraints)`，它返回一个 `Problem` 对象。

`Problem` 对象具有函数 solve()，它返回解的最小值，即投资组合的最小方差。

它还会更新向量 $\mathbf{x}$。

我们可以使用 `x.value` 查看得出最小投资组合方差的 $x\_A$ 和 $x\_B$ 值。

In [ ]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    
    return None

project_tests.test_get_optimal_weights(get_optimal_weights)

## 优化的投资组合
使用 `get_optimal_weights` 函数生成不会再平衡的最优 ETF 权重。我们可以传入整个历史数据的协方差。还需要传入一组指数权重。我们将使用一段时间的平均指数权重。

In [ ]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

设定 ETF 权重后，将其与指数权重进行比较。运行以下单元格，以计算 ETF 收益率并与指数收益率进行比较。

In [ ]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

## 过段时间后再平衡投资组合
单个优化 ETF 投资组合在整个历史记录中使用了相同的权重。这个权重对于整个时期来说可能不是最优权重。我们对相同时期的投资组合进行再平衡，而不使用相同的权重。实现 `rebalance_portfolio` 以再平衡投资组合。

每隔 n 天（由 `shift_size` 确定）再平衡投资组合。再平衡时，你应该查看过去特定天数的数据（由 `chunk_size` 表示）。根据这些数据使用 `get_optimal_weights` 和 `get_covariance_returns` 计算最优权重。

In [ ]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    
    return None

project_tests.test_rebalance_portfolio(rebalance_portfolio)

运行以下单元格以使用 `rebalance_portfolio` 再平衡投资组合。

In [ ]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

## 投资组合周转率
再平衡投资组合后，我们需要一个指标来衡量再平衡投资组合的成本。实现 `get_portfolio_turnover` 以计算年度投资组合周转率。我们将使用在教室中用到的公式：

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} \* NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ 其中 $ x_{t,n} $ 是股权 $ n $ 在时间 $ t $ 的权重。

$ SumTotalTurnover $ 是 $ \sum \left | x_{t_1,n} - x_{t\_2,n} \right | $ 的另一种书写方式。

In [ ]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    
    return None

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

运行以下单元格以使用  `get_portfolio turnover` 获取投资组合周转率。

In [ ]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

搞定了！你已经在第一部分构建了一个 smart beta 投资组合，并且在第二部分优化了该投资组合。现在可以提交项目了。

## 提交项目
完成项目后，就可以提交了。请点击右下角的提交按钮。我们的审阅专家将对项目打分（通过或不通过）并提供反馈。在等待反馈期间，你可以继续学习下个部分。

[1]:	https://pandas.pydata.org/
[2]:	http://www.numpy.org/
[3]:	https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html
[4]:	https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html
[5]:	https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm